In [ ]:
!pip install -q openai langchain langchain-openai

In [ ]:
import os
import json
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [ ]:
OPENROUTER_API_KEY = "Your Token Here"
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
MODEL_NAME = "moonshotai/kimi-k2:free"

In [ ]:
client = OpenAI(
    base_url=OPENROUTER_BASE_URL,
    api_key=OPENROUTER_API_KEY,
)

llm = ChatOpenAI(
    model=MODEL_NAME,
    base_url=OPENROUTER_BASE_URL,
    api_key=OPENROUTER_API_KEY,
)

print("Successfully initialized clients to use OpenRouter.")

Successfully initialized clients to use OpenRouter.


In [ ]:
def run_example_1_simple_question():
    """1. Makes a simple API call with a sample question."""
    print("\n--- Example 1: Simple API Call ---")
    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "What is the largest planet in our solar system?"}
            ]
        )
        answer = response.choices[0].message.content
        print(f"Question: What is the largest planet in our solar system?")
        print(f"Answer: {answer}")
    except Exception as e:
        print(f"An error occurred during the API call: {e}")

In [ ]:
run_example_1_simple_question()


--- Example 1: Simple API Call ---
Question: What is the largest planet in our solar system?
Answer: The largest planet in our solar system is **Jupiter**.


In [ ]:
biography_text = """
Marie Curie, born on November 7, 1867, was a Polish and naturalized-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person and the only woman to win the Nobel Prize twice, and the only person to win the Nobel Prize in two different scientific fields.
Her husband, Pierre Curie, was a co-winner on her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She died in 1934 at the age of 66.
"""

In [ ]:
def run_example_2_question_with_context():
    """2. Asks a question based on a provided biography."""
    print("\n--- Example 2: Question with Provided Context ---")
    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "You are an expert researcher. Answer the user's question based on the provided text."},
                {"role": "user", "content": f"Based on this biography, in what year was Marie Curie born?\n\nBiography:\n{biography_text}"}
            ]
        )
        answer = response.choices[0].message.content
        print(f"Question: Based on the biography, in what year was Marie Curie born?")
        print(f"Answer: {answer}")
    except Exception as e:
        print(f"An error occurred during the API call: {e}")

In [ ]:
run_example_2_question_with_context()


--- Example 2: Question with Provided Context ---
Question: Based on the biography, in what year was Marie Curie born?
Answer: Marie Curie was born in 1867.


In [ ]:
def run_example_3_langchain_sequential_pipeline():
    """3. Uses a LangChain pipeline with successive calls to build a JSON object."""
    print("\n--- Example 3: LangChain Sequential Pipeline for JSON ---")
    print("This method is illustrative but less efficient as it makes multiple calls.")
    try:
        prompt_template = ChatPromptTemplate.from_template(
            "Based on the text below, what is the person's {attribute}?\n\nText: {context}"
        )
        parser = StrOutputParser()

        chain_name = prompt_template | llm | parser
        chain_birth_year = prompt_template | llm | parser
        chain_gender = prompt_template | llm | parser

        print("Requesting name...")
        name = chain_name.invoke({"attribute": "full name", "context": biography_text})

        print("Requesting birth year...")
        birth_year_str = chain_birth_year.invoke({"attribute": "birth year", "context": biography_text})
        birth_year = int(''.join(filter(str.isdigit, birth_year_str)))


        print("Requesting gender...")
        gender = chain_gender.invoke({"attribute": "gender", "context": biography_text})

        person_json = {
            "name": name.strip(),
            "birth_year": birth_year,
            "gender": gender.strip()
        }

        print("\nFinal JSON object:")
        print(json.dumps(person_json, indent=2))

    except Exception as e:
        print(f"An error occurred during the LangChain pipeline: {e}")

In [ ]:
run_example_3_langchain_sequential_pipeline()


--- Example 3: LangChain Sequential Pipeline for JSON ---
This method is illustrative but less efficient as it makes multiple calls.
Requesting name...
Requesting birth year...
Requesting gender...

Final JSON object:
{
  "name": "Marie Sk\u0142odowska Curie (birth name: Maria Salomea Sk\u0142odowska)",
  "birth_year": 1867,
  "gender": "The person is female."
}


In [ ]:
def run_example_4_langchain_structured_output():
    """4. Uses a single call with a JSON output parser for efficient, structured data."""
    print("\n--- Example 4: LangChain with Structured Output Parser (Single Call) ---")
    print("This is the recommended, efficient method for structured data extraction.")

    try:
        class PersonInfo(BaseModel):
            name: str = Field(description="The full name of the person.")
            birth_year: int = Field(description="The year the person was born.")
            gender: str = Field(description="The gender of the person.")

        parser = JsonOutputParser(pydantic_object=PersonInfo)

        prompt = ChatPromptTemplate.from_template(
            "Answer the user query based on the provided context.\n"
            "{format_instructions}\n"
            "Context: {context}\n"
            "Query: Extract the required information for the person described."
        )

        chain = prompt | llm | parser

        print("Invoking chain to extract structured data in a single call...")
        result_dict = chain.invoke({
            "context": biography_text,
            "format_instructions": parser.get_format_instructions()
        })

        print("\nFinal parsed dictionary:")
        print(result_dict)
        print(f"\nType of result: {type(result_dict)}")
        print(f"Accessing data: {result_dict['name']} was born in {result_dict['birth_year']}.")

    except Exception as e:
        print(f"An error occurred during the structured output chain: {e}")

In [ ]:
run_example_4_langchain_structured_output()


--- Example 4: LangChain with Structured Output Parser (Single Call) ---
This is the recommended, efficient method for structured data extraction.
Invoking chain to extract structured data in a single call...

Final parsed dictionary:
{'name': 'Marie Skłodowska Curie', 'birth_year': 1867, 'gender': 'female'}

Type of result: <class 'dict'>
Accessing data: Marie Skłodowska Curie was born in 1867.
